In [33]:
# %%
from langchain_groq import ChatGroq

# Initialize the LLM with given API key
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

# %%
from langchain_community.document_loaders import WebBaseLoader

# Load job page data
loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)

# %%
from langchain_core.prompts import PromptTemplate

# Define a prompt to extract job information in JSON format
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)

# Run the extraction chain
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

# %%
from langchain_core.output_parsers import JsonOutputParser

# Parse the JSON result from the extraction
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

# %%
import pandas as pd

# Load portfolio data from CSV
df = pd.read_csv("my_portfolio.csv")
print(df)

# %%
import uuid
import chromadb

# Initialize the ChromaDB client
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

# Add data to the collection only if it’s empty
if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],  # Ensure documents is a list
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

# %%
# Ensure `job` is correctly defined by selecting the first job if `json_res` is a list
if isinstance(json_res, list) and json_res:
    job = json_res[0]  # Select the first job in the list
else:
    job = json_res  # Use it directly if it’s already a dictionary

# Verify `job` contains 'skills' and it’s in the correct format
if 'skills' not in job or not isinstance(job['skills'], list):
    print("Job data is missing 'skills' or 'skills' is not a list.")
    job['skills'] = []  # Default to empty list to prevent errors

# Query the collection for relevant links based on job skills
links = []
if job['skills']:  # Proceed only if there are skills to query
    links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

# Handle case where no links are found
if not links:
    print("No relevant links found in the collection.")
    link_list = []  # Set an empty list if no results are found
else:
    # Extract only the 'links' part from the results for use in the email prompt
    link_list = [link['links'] for link in links]
print(link_list)

# %%
# Define a prompt template for generating the cold email
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase AtliQ's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

# Run the email generation chain
chain_email = prompt_email | llm
res = chain_email.invoke({
    "job_description": str(job), 
    "link_list": link_list
})
print(res.content)


Neil Armstrong. He stepped onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission.


Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN U